## Authenticate to Colab and Cloud


In [0]:
from google.colab import auth

auth.authenticate_user()

## Authenticate to Earth Engine and initialize

In [0]:
# Import the Earth Engine API and initialize it.
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Test the Earth Engine installation



In [0]:
# Test the earthengine command by getting help on upload.
!earthengine upload image -h

## Test the TensorFlow installation

upgraded to TF2.0

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
print(tf.__version__)

# Import other libraries

In [0]:
import numpy as np
import folium
print(folium.__version__)

## Prepare Landsat 8 imagery

First, make a mosaic of Landsat 8 surface reflectance imagery. We will analyze the full time series for 2017

In [0]:
#date sequence
Date_Start = ee.Date('2017-01-01')
Date_End = ee.Date('2017-12-31')
dstep = 16

n_day = Date_End.difference(Date_Start,'day').round()
dates = ee.List.sequence(0,n_day,dstep)
def make_datelist(n):
  return Date_Start.advance(n,'day')

dates = dates.map(make_datelist)

In [0]:
# Change the following two lines to use your own training data.
labels = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')
label = 'landcover'

# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','nd']
# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')


In [0]:
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).multiply(10000)
  return image.addBands(ndvi)

#to fill nulls with previous non null record
def gap_fill(image, listb):
  current = image.unmask(-999)
  previous = ee.Image(ee.List(listb).get(-1))
  new = current.where(current.eq(-999), previous).set('system:time_start', image.get('system:time_start'))
  return ee.List(listb).add(new)

# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

def getl8 (dlist):
  #extract dates
  dat = ee.Date(dlist)
  dat2 = dat.advance(dstep,'day')
  dat_str = dat2.format()

  #summarize l8 between dates
  imageCol_gaps = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
  .filterBounds(labels)\
  .filterDate(dat, dat.advance(dstep,'day'))\
  .map(addNDVI)\
  .map(maskL8sr)\
  .median()\
  .set('system:time_start', dat_str)
       
  return imageCol_gaps

ilist = dates.map(getl8)
imageCol_gaps = ee.ImageCollection(ilist)



In [0]:
#fill non null value in series
first_im = imageCol_gaps\
  .reduce(ee.Reducer.firstNonNull())\
  .rename(bands)\
  .set({'system:index': 'first'})
first = ee.List([first_im])

#fill gaps then drop first image
imageCol = ee.ImageCollection(ee.List(imageCol_gaps.iterate(gap_fill, first)))\
  .filter(ee.Filter.neq('system:index', 'first'))

#flatten each band
imageCol_all = imageCol\
  .toArrayPerBand()

tsLength = imageCol.size().getInfo()
image = imageCol_all

## sample pixel values at labeled points. 

In [635]:
# Sample the image at the points and add a random column.
sample = image.sampleRegions(
  collection=labels, properties=[label], scale=30).randomColumn()

# Partition the sample approximately 70-30.
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))

from pprint import pprint

# Print the first couple points to verify.
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

{'training': {'geometry': None,
              'id': '00000000000000000003_0',
              'properties': {'B2': [0.06350000202655792,
                                    0.06350000202655792,
                                    0.06350000202655792,
                                    0.06350000202655792,
                                    0.06350000202655792,
                                    0.06350000202655792,
                                    0.06350000202655792,
                                    0.09510000050067902,
                                    0.10300000011920929,
                                    0.10300000011920929,
                                    0.11100000143051147,
                                    0.11100000143051147,
                                    0.08760000020265579,
                                    0.08269999921321869,
                                    0.08269999921321869,
                                    0.08749999850988388,
          

## Export the training and testing data


In [636]:
# REPLACE WITH YOUR BUCKET!
outputBucket = 'ee-tf-example'

# Make sure the bucket exists.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + outputBucket) 
    else 'Output Cloud Storage bucket does not exist.')

Found Cloud Storage bucket.


In [0]:
# Names for output files.
trainFilePrefix = 'Training_demo_cnn'
testFilePrefix = 'Testing_demo_cnn'

# This is list of all the properties we want to export.
featureNames = list(bands)
featureNames.append(label)


In [0]:
# Create the tasks.
trainingTask = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix=trainFilePrefix,
  bucket=outputBucket,
  fileFormat='TFRecord',
  selectors=featureNames)

testingTask = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  fileNamePrefix=testFilePrefix,
  bucket=outputBucket,
  fileFormat='TFRecord',
  selectors=featureNames)

In [0]:
# Start the tasks.
trainingTask.start()
testingTask.start()

In [620]:
# Print all tasks.
print(ee.batch.Task.list())

# Poll the training task until it's done.
import time 
while trainingTask.active():
  print('Polling for task (id: {}).'.format(trainingTask.id))
  time.sleep(30)
print('Done with training export.')

[<Task EXPORT_FEATURES: Testing Export (READY)>, <Task EXPORT_FEATURES: Training Export (RUNNING)>, <Task INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/glennwithtwons/Classified_pixel_demo_2021" (COMPLETED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/glennwithtwons/Classified_pixel_demo_2020" (COMPLETED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_IMAGE: Image Export (FAILED)>, <Task EXPORT_IMAGE: Image Export (FAILED)>, <Task EXPORT_IMAGE: Image Export (FAILED)>, <Task EXPORT_IMAGE: Image Export (FAILED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/glennwithtwons/Classified_pixel_demo99" (COMPLETED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_IMAGE: Image Export (FAILED)>, <T

### Check existence of the exported files


In [641]:
fileNameSuffix = '.tfrecord.gz'
trainFilePath = 'gs://' + outputBucket + '/' + trainFilePrefix + fileNameSuffix
testFilePath = 'gs://' + outputBucket + '/' + testFilePrefix + fileNameSuffix

print('Found training file.' if tf.io.gfile.exists(trainFilePath) 
    else 'No training file found.')
print('Found testing file.' if tf.io.gfile.exists(testFilePath) 
    else 'No testing file found.')

Found training file.
Found testing file.


## Export the imagery as tfrecord

In [0]:
#define bands and dim
patchdim = 256
fmap = dict(zip(list(bands), np.repeat(tsLength,len(bands)).tolist()))

In [0]:
imageFilePrefix = 'Image_pixel_demo_2020'
# Specify patch and file dimensions.
imageExportFormatOptions = {
  'patchDimensions': [patchdim, patchdim],
  'maxFileSize': 1048576000,
  'compressed': True,
  'tensorDepths': fmap
}

# Export imagery in this region.
exportRegion =  ee.Geometry.Polygon(\
        [[[-122.49712825452804, 37.72807005959367],\
          [-122.49712825452804, 37.65471739519173],\
          [-122.41953731214522, 37.65471739519173],\
          [-122.41953731214522, 37.72807005959367]]])
#exportRegion = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00])

# Setup the task
imageTask = ee.batch.Export.image.toCloudStorage(
  image=image,
  description='Image Export',
  fileNamePrefix=imageFilePrefix,
  bucket=outputBucket,
  scale=30,
  fileFormat='TFRecord',
  region=exportRegion.toGeoJSON()['coordinates'],
  formatOptions=imageExportFormatOptions,
)

In [0]:
# Start the task.
imageTask.start()

In [645]:
imageTask.status()

{'creation_timestamp_ms': 1581081842472,
 'description': 'Image Export',
 'id': '2HMHL2E7F5UHIYAPQWDVDYNY',
 'name': 'projects/earthengine-legacy/operations/2HMHL2E7F5UHIYAPQWDVDYNY',
 'start_timestamp_ms': 1581081852940,
 'state': 'RUNNING',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1581081853071}

### Monitor task progress

Before making predictions, we need the image export to finish, so block until it does.  This might take a few minutes...

In [0]:
while imageTask.active():
  print('Polling for task (id: {}).'.format(imageTask.id))
  time.sleep(30)
print('Done with image export.')

# Data preparation and pre-processing

In [646]:
# Create a dataset from the TFRecord file in Cloud Storage.
trainDataset = tf.data.TFRecordDataset(trainFilePath, compression_type='GZIP')
# Print the first record to check.
print(iter(trainDataset).next())

tf.Tensor(b"\n\xef\x05\nf\n\x02B2\x12`\x12^\n\\J\x0c\x82=J\x0c\x82=J\x0c\x82=J\x0c\x82=J\x0c\x82=J\x0c\x82=J\x0c\x82=\xca\xc3\xc2=\xaa\xf1\xd2=\xaa\xf1\xd2=\xf8S\xe3=\xf8S\xe3=\xa1g\xb3=\x9e^\xa9=\x9e^\xa9=33\xb3=33\xb3=\xe5a\xa1=\xe5a\xa1=\xbe0\x99=\xbe0\x99=\xba\xda\x8a=\xba\xda\x8a=\nf\n\x02B3\x12`\x12^\n\\\xa0\x1a\xaf=\xa0\x1a\xaf=\xa0\x1a\xaf=\xa0\x1a\xaf=\xa0\x1a\xaf=\xa0\x1a\xaf=\xa0\x1a\xaf=\xb2\x9d\xef=o\xf0\x05>o\xf0\x05>\xdd$\x06>\xdd$\x06>\xa7\xe8\xc8=\xf3\x1f\xd2=\xf3\x1f\xd2=\x85|\xd0=\x85|\xd0=W\xec\xaf=W\xec\xaf=~\x8c\xb9=~\x8c\xb9=\x9c\xa2\xa3=\x9c\xa2\xa3=\nf\n\x02B4\x12`\x12^\n\\\xc5\xfe\xb2=\xc5\xfe\xb2=\xc5\xfe\xb2=\xc5\xfe\xb2=\xc5\xfe\xb2=\xc5\xfe\xb2=\xc5\xfe\xb2=\xb57\xf8=\xf1c\x0c>\xf1c\x0c>\xcf\xf7\x13>\xcf\xf7\x13>\xf5\xb9\xda=\xf8S\xe3=\xf8S\xe3=\xab\xcf\xd5=\xab\xcf\xd5=\xee|\xbf=\xee|\xbf=\xf1\xf4\xca=\xf1\xf4\xca=\xc0[\xa0=\xc0[\xa0=\nf\n\x02B5\x12`\x12^\n\\\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\xcc\xcc=\xe3\xc

In [647]:
# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=[tsLength], dtype=tf.float32),
  tf.io.FixedLenFeature(shape=[tsLength], dtype=tf.float32),
  tf.io.FixedLenFeature(shape=[tsLength], dtype=tf.float32),
  tf.io.FixedLenFeature(shape=[tsLength], dtype=tf.float32),
  tf.io.FixedLenFeature(shape=[tsLength], dtype=tf.float32),
  tf.io.FixedLenFeature(shape=[tsLength], dtype=tf.float32),
  tf.io.FixedLenFeature(shape=[tsLength], dtype=tf.float32),
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
]

# Dictionary with names as keys, features as values.
featuresDict = dict(zip(featureNames, columns))

pprint(featuresDict)

{'B2': FixedLenFeature(shape=[23], dtype=tf.float32, default_value=None),
 'B3': FixedLenFeature(shape=[23], dtype=tf.float32, default_value=None),
 'B4': FixedLenFeature(shape=[23], dtype=tf.float32, default_value=None),
 'B5': FixedLenFeature(shape=[23], dtype=tf.float32, default_value=None),
 'B6': FixedLenFeature(shape=[23], dtype=tf.float32, default_value=None),
 'B7': FixedLenFeature(shape=[23], dtype=tf.float32, default_value=None),
 'landcover': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'nd': FixedLenFeature(shape=[23], dtype=tf.float32, default_value=None)}


## Parse the dataset


In [0]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.
  
  Returns: 
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
  labels = parsed_features.pop(label)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsedDataset = trainDataset.map(parse_tfrecord, num_parallel_calls=5)

# How many classes there are in the model.
nClasses = 3

inputDataset = parsedDataset


In [0]:

# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def toTuple(dictionary, label):
  return tf.transpose(list(dictionary.values())), tf.one_hot(indices=label, depth=nClasses)

# Repeat the input dataset as many times as necessary in batches of 10.
inputDataset = inputDataset.map(toTuple).repeat().batch(10)
#inputDataset = inputDataset.map(toTuple).shuffle(buffer_size=20, seed=42).batch(10)


## Create the Keras model
 architecture from https://github.com/charlotte-pel/igarss2019-dl4sits

In [650]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, BatchNormalization


# Define the layers in the model.
model = Sequential([
  Conv1D(filters=32, kernel_size=5, padding = 'same', activation='relu',input_shape=(tsLength,len(bands))),
 # BatchNormalization(),
 # Dropout(0.2),
  Conv1D(filters=32, kernel_size=5, padding = 'same', activation='relu'),
 # BatchNormalization(),
 # Dropout(0.2),
  Flatten(),
  Dense(256, activation='relu'),
#  BatchNormalization(),
  Dropout(0.2),
  Dense(nClasses, activation='softmax')
])

# Compile the model with the specified loss function.
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model to the training data.
# Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.
model.fit(x=inputDataset, epochs=5,steps_per_epoch=100)


Train for 100 steps
Epoch 1/5
100/100 [==============================] - 2s 22ms/step - loss: 0.2608 - accuracy: 0.7950
Epoch 2/5
100/100 [==============================] - 2s 18ms/step - loss: 0.0175 - accuracy: 0.8536
Epoch 3/5
100/100 [==============================] - 2s 18ms/step - loss: 0.0026 - accuracy: 0.8608
Epoch 4/5
100/100 [==============================] - 2s 17ms/step - loss: 0.0011 - accuracy: 0.8592
Epoch 5/5
100/100 [==============================] - 2s 20ms/step - loss: 5.9078e-04 - accuracy: 0.8560


## Check model accuracy on the test set


In [651]:
testDataset = (
  tf.data.TFRecordDataset(testFilePath, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(toTuple)
    .batch(10)
)

model.evaluate(testDataset, steps=100)

  1/100 [..............................] - ETA: 25s - loss: 0.0023 - accuracy: 0.6800WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 100 batches). You may need to use the repeat() function when building your dataset.


[2.356533296165253e-05, 0.7866667]

# Use the trained model to classify an image from Earth Engine


## Find the image files and JSON mixer file in Cloud Storage


In [652]:
# Get a list of all the files in the output bucket.
filesList = !gsutil ls 'gs://'{outputBucket}
# Get only the files generated by the image export.
exportFilesList = [s for s in filesList if imageFilePrefix in s]

# Get the list of image files and the JSON mixer file.
imageFilesList = []
jsonFile = None
for f in exportFilesList:
  if f.endswith('.tfrecord.gz'):
    imageFilesList.append(f)
  elif f.endswith('.json'):
    jsonFile = f

# Make sure the files are in the right order.
print(imageFilesList.sort())


None


In [653]:

pprint(imageFilesList)
print(jsonFile)

['gs://ee-tf-example/Image_pixel_demo_2020.tfrecord.gz']
gs://ee-tf-example/Image_pixel_demo_2020.json


## Read the JSON mixer file

The mixer contains metadata and georeferencing information for the exported patches, each of which is in a different file.  Read the mixer to get some information needed for prediction.

In [654]:
import json

# Load the contents of the mixer file to a JSON object.
jsonText = !gsutil cat {jsonFile}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(jsonText.nlstr)
pprint(mixer)

{'patchDimensions': [256, 256],
 'patchesPerRow': 1,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            -122.49714675144715,
                                            0.0,
                                            -0.00026949458523585647,
                                            37.72816395467896]},
                'crs': 'EPSG:4326'},
 'totalPatches': 1}


## Read the image files into a dataset


In [0]:
# Get relevant info from the JSON mixer file.
PATCH_WIDTH = mixer['patchDimensions'][0]
PATCH_HEIGHT = mixer['patchDimensions'][1]
PATCHES = mixer['totalPatches']
PATCH_DIMENSIONS_FLAT = [PATCH_WIDTH * PATCH_HEIGHT,tsLength]

# Note that the tensors are in the shape of a patch, one patch for each band.
imageColumns = [
  tf.io.FixedLenFeature(shape=PATCH_DIMENSIONS_FLAT, dtype=tf.float32) 
    for k in bands
]

# Parsing dictionary.
imageFeaturesDict = dict(zip(bands, imageColumns))

# Note that you can make one dataset from many files by specifying a list.
imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')


In [656]:
imageFeaturesDict

{'B2': FixedLenFeature(shape=[65536, 23], dtype=tf.float32, default_value=None),
 'B3': FixedLenFeature(shape=[65536, 23], dtype=tf.float32, default_value=None),
 'B4': FixedLenFeature(shape=[65536, 23], dtype=tf.float32, default_value=None),
 'B5': FixedLenFeature(shape=[65536, 23], dtype=tf.float32, default_value=None),
 'B6': FixedLenFeature(shape=[65536, 23], dtype=tf.float32, default_value=None),
 'B7': FixedLenFeature(shape=[65536, 23], dtype=tf.float32, default_value=None),
 'nd': FixedLenFeature(shape=[65536, 23], dtype=tf.float32, default_value=None)}

In [0]:
# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, imageFeaturesDict)

# Parse the data into tensors, one long tensor per patch.
imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)


In [0]:
# Break our long tensors into many little ones.
imageDataset = imageDataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)


In [0]:
# Turn the dictionary in each record into a tuple without a label.
imageDataset = imageDataset.map(
  lambda dataDict: (tf.transpose(list(dataDict.values())), )
)

In [0]:
# Turn each patch into a batch.
imageDataset = imageDataset.batch(PATCH_WIDTH * PATCH_HEIGHT)

## Generate predictions for the image pixels

To get predictions in each pixel, run the image dataset through the trained model using `model.predict()`.  Print the first prediction to see that the output is a list of the three class probabilities for each pixel.  Running all predictions might take a while.

In [661]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model.predict(imageDataset, steps=1, verbose=1)

# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])

1/1 [==============================] - 4s 4s/step
[5.6373872e-05 9.9854076e-01 1.4028219e-03]


## Write the predictions to a TFRecord file


In [662]:
outputImageFile = 'gs://' + outputBucket + '/Classified_pixel_demo_2022.TFRecord'
print('Writing to file ' + outputImageFile)

Writing to file gs://ee-tf-example/Classified_pixel_demo_2022.TFRecord


In [663]:
# Instantiate the writer.
writer = tf.io.TFRecordWriter(outputImageFile)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[], [], [], []]
curPatch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction))
  patch[1].append(prediction[0])
  patch[2].append(prediction[1])
  patch[3].append(prediction[2])
  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == PATCH_WIDTH * PATCH_HEIGHT):
    print('Done with patch ' + str(curPatch) + ' of ' + str(PATCHES) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          'bareProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          'vegProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2])),
          'waterProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[3])),
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of class ids
    writer.write(example.SerializeToString())
    patch = [[], [], [], []]
    curPatch += 1

writer.close()

Done with patch 1 of 1...


# Upload the classifications to an Earth Engine asset

## Verify the existence of the predictions file

In [0]:
!gsutil ls -l {outputImageFile}

## Upload the classified image to Earth Engine


In [664]:
# REPLACE WITH YOUR USERNAME:
USER_NAME = 'glennwithtwons'
outputAssetID = 'users/' + USER_NAME + '/Classified_pixel_demo_2020'
print('Writing to ' + outputAssetID)

Writing to users/glennwithtwons/Classified_pixel_demo_2020


In [0]:
# Start the upload.
!earthengine upload image --asset_id={outputAssetID} {outputImageFile} {jsonFile}

In [0]:
ee.batch.Task.list()

## View the ingested asset

Display the vector of class probabilities as an RGB image with colors corresponding to the probability of bare, vegetation, water in a pixel.  Also display the winning class using the same color palette.

In [666]:
predictionsImage = ee.Image(outputAssetID)

predictionVis = {
  'bands': 'prediction',
  'min': 0,
  'max': 2,
  'palette': ['red', 'green', 'blue']
}
probabilityVis = {'bands': ['bareProb', 'vegProb', 'waterProb']}

predictionMapid = predictionsImage.getMapId(predictionVis)
probabilityMapid = predictionsImage.getMapId(probabilityVis)

map = folium.Map(location=[38., -122.5])
folium.TileLayer(
  tiles=predictionMapid['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='prediction',
).add_to(map)
folium.TileLayer(
  tiles=probabilityMapid['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='probability',
).add_to(map)
map.add_child(folium.LayerControl())
map